<a href="https://colab.research.google.com/github/atharvavyas1/Finance-N8N-project/blob/main/YahooFinanceRSS_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install feedparser pandas requests beautifulsoup4

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 1.6 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=1f83f826de61537f6a5b20d67db5c7847edd46a3ede9bc0c43c256fcd2306c21
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [ ]:
import feedparser
import pandas as pd
from datetime import datetime
import json
import time

class YahooFinanceRSSScraper:
    """
    Scrape Yahoo Finance news using RSS feeds
    RSS feeds are public and don't require authentication
    """

    def __init__(self):
        self.base_rss_url = "https://feeds.finance.yahoo.com/rss/2.0/headline?s={}&region=US&lang=en-US"

    def get_news_for_ticker(self, ticker, verbose=True):
        """
        Get news for a single ticker symbol

        Args:
            ticker (str): Stock ticker symbol (e.g., 'AAPL')
            verbose (bool): Print status messages

        Returns:
            list: List of news articles
        """
        articles = []

        if verbose:
            print(f"Fetching RSS feed for {ticker}...")

        try:
            # Construct the RSS feed URL
            feed_url = self.base_rss_url.format(ticker.upper())

            # Parse the RSS feed
            feed = feedparser.parse(feed_url)

            # Check if feed was successfully parsed
            if feed.bozo:
                print(f"  Warning: Feed parsing issues for {ticker} - {feed.bozo_exception}")

            if feed.entries:
                for entry in feed.entries:
                    # Extract and clean the data
                    article = {
                        'ticker': ticker.upper(),
                        'title': entry.get('title', '').strip(),
                        'link': entry.get('link', '').strip(),
                        'description': entry.get('summary', '').strip(),
                        'guid': entry.get('guid', '').strip(),
                        'published': entry.get('published', ''),
                    }

                    # Parse the publication date to a more readable format
                    if hasattr(entry, 'published_parsed') and entry.published_parsed:
                        try:
                            article['published_datetime'] = datetime(*entry.published_parsed[:6])
                            article['published_formatted'] = article['published_datetime'].strftime('%Y-%m-%d %H:%M:%S')
                        except:
                            article['published_datetime'] = None
                            article['published_formatted'] = article['published']
                    else:
                        article['published_datetime'] = None
                        article['published_formatted'] = article['published']

                    # Add scraping metadata
                    article['scraped_at'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

                    articles.append(article)

                if verbose:
                    print(f"  ✓ Found {len(feed.entries)} articles for {ticker}")
            else:
                if verbose:
                    print(f"  ⚠ No articles found for {ticker}")

        except Exception as e:
            print(f"  ✗ Error fetching RSS for {ticker}: {e}")

        return articles

    def get_news_for_multiple_tickers(self, tickers, delay=0.5, verbose=True):
        """
        Get news for multiple ticker symbols

        Args:
            tickers (list): List of ticker symbols
            delay (float): Delay in seconds between requests
            verbose (bool): Print status messages

        Returns:
            dict: Dictionary with tickers as keys and article lists as values
        """
        all_news = {}
        all_articles = []

        # Convert single ticker to list
        if isinstance(tickers, str):
            tickers = [tickers]

        print(f"\nFetching news for {len(tickers)} tickers...")
        print("=" * 50)

        for i, ticker in enumerate(tickers, 1):
            # Get news for this ticker
            articles = self.get_news_for_ticker(ticker, verbose)

            # Store results
            all_news[ticker.upper()] = articles
            all_articles.extend(articles)

            # Add delay between requests (except for last ticker)
            if i < len(tickers) and delay > 0:
                time.sleep(delay)

        # Print summary
        print("=" * 50)
        print(f"\nSummary:")
        print(f"  Total tickers processed: {len(tickers)}")
        print(f"  Total articles found: {len(all_articles)}")

        for ticker, articles in all_news.items():
            print(f"  {ticker}: {len(articles)} articles")

        return all_news, all_articles

    def save_to_csv(self, articles, filename='yahoo_rss_news.csv'):
        """Save articles to CSV file"""
        if not articles:
            print("No articles to save")
            return

        # Convert to DataFrame
        df = pd.DataFrame(articles)

        # Reorder columns for better readability
        columns_order = ['ticker', 'title', 'description', 'published_formatted',
                        'link', 'published', 'published_datetime', 'guid', 'scraped_at']

        # Only include columns that exist
        columns_to_save = [col for col in columns_order if col in df.columns]
        df = df[columns_to_save]

        # Save to CSV
        df.to_csv(filename, index=False, encoding='utf-8')
        print(f"\n✓ Saved {len(articles)} articles to {filename}")

        return df

    def save_to_json(self, data, filename='yahoo_rss_news.json'):
        """Save articles to JSON file"""
        if not data:
            print("No data to save")
            return

        # Convert datetime objects to strings for JSON serialization
        def json_serializer(obj):
            if isinstance(obj, datetime):
                return obj.strftime('%Y-%m-%d %H:%M:%S')
            return str(obj)

        # Save to JSON
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False, default=json_serializer)

        article_count = len(data) if isinstance(data, list) else sum(len(v) for v in data.values())
        print(f"✓ Saved {article_count} articles to {filename}")

    def display_articles(self, articles, max_display=5, truncate_desc=True, max_desc_length=None):
        """
        Display articles in a readable format

        Args:
            articles (list): List of articles to display
            max_display (int): Maximum number of articles to show
            truncate_desc (bool): Whether to truncate descriptions
            max_desc_length (int): Maximum description length (None for no limit)
        """
        if not articles:
            print("No articles to display")
            return

        print(f"\nDisplaying {min(len(articles), max_display)} of {len(articles)} articles:")
        print("=" * 80)

        for i, article in enumerate(articles[:max_display], 1):
            print(f"\n{i}. {article['title']}")
            print(f"   Ticker: {article['ticker']}")
            print(f"   Published: {article.get('published_formatted', article.get('published', 'Unknown'))}")

            # Handle description display
            desc = article.get('description', '')
            if desc:
                if truncate_desc and max_desc_length and len(desc) > max_desc_length:
                    desc = desc[:max_desc_length] + "..."
                print(f"   Summary: {desc}")

            # Handle link display
            link = article['link']
            if truncate_desc and len(link) > 80:
                print(f"   Link: {link[:80]}...")
            else:
                print(f"   Link: {link}")

        print("=" * 80)


# ====================
# EXAMPLE USAGE
# ====================

def main():
    """Main function to demonstrate usage"""

    # Initialize the scraper
    scraper = YahooFinanceRSSScraper()

    # Example 1: Single ticker
    print("\n" + "="*50)
    print("EXAMPLE 1: Single Ticker (AAPL)")
    print("="*50)

    apple_news = scraper.get_news_for_ticker('AAPL')
    # Display with full descriptions (no truncation)
    scraper.display_articles(apple_news, max_display=3, truncate_desc=False)

    # Example 2: Multiple tickers
    print("\n" + "="*50)
    print("EXAMPLE 2: Multiple Tickers")
    print("="*50)

    tickers = ['AAPL', 'GOOGL', 'MSFT', 'TSLA', 'AMZN', 'META', 'NVDA']
    all_news, all_articles = scraper.get_news_for_multiple_tickers(tickers)

    # Save results
    print("\nSaving results...")
    scraper.save_to_csv(all_articles)
    scraper.save_to_json(all_news)

    # Example 3: Filter by date
    print("\n" + "="*50)
    print("EXAMPLE 3: Recent Articles (Last 24 hours)")
    print("="*50)

    from datetime import timedelta
    cutoff_date = datetime.now() - timedelta(days=1)

    recent_articles = [
        article for article in all_articles
        if article.get('published_datetime') and article['published_datetime'] > cutoff_date
    ]

    print(f"Found {len(recent_articles)} articles from the last 24 hours")
    # Display recent articles with full content
    scraper.display_articles(recent_articles, max_display=3, truncate_desc=False)

    # Example 4: Search for specific keywords in titles
    print("\n" + "="*50)
    print("EXAMPLE 4: Keyword Search")
    print("="*50)

    keyword = "earnings"
    matching_articles = [
        article for article in all_articles
        if keyword.lower() in article['title'].lower()
    ]

    print(f"Found {len(matching_articles)} articles containing '{keyword}'")
    # Display matching articles with partial truncation
    scraper.display_articles(matching_articles, max_display=3, max_desc_length=300)

    # Create a summary DataFrame
    print("\n" + "="*50)
    print("Creating Summary Report...")
    print("="*50)

    if all_articles:
        df = pd.DataFrame(all_articles)

        # Summary statistics
        print(f"\nNews Summary Statistics:")
        print(f"  Total articles: {len(df)}")
        print(f"  Unique tickers: {df['ticker'].nunique()}")
        print(f"\nArticles per ticker:")
        print(df['ticker'].value_counts())

        # Save summary to Excel (optional)
        try:
            with pd.ExcelWriter('yahoo_news_summary.xlsx') as writer:
                df.to_excel(writer, sheet_name='All News', index=False)
                df['ticker'].value_counts().to_excel(writer, sheet_name='Summary')
            print("\n✓ Saved Excel summary to yahoo_news_summary.xlsx")
        except:
            pass


if __name__ == "__main__":
    # Run the main function
    main()

    # Quick start for custom usage
    print("\n" + "="*50)
    print("QUICK START CODE:")
    print("="*50)
    print("""
# Minimal code to get started:
from yahoo_rss_scraper import YahooFinanceRSSScraper

scraper = YahooFinanceRSSScraper()

# Single ticker
news = scraper.get_news_for_ticker('AAPL')

# Multiple tickers
tickers = ['AAPL', 'GOOGL', 'MSFT']
all_news, all_articles = scraper.get_news_for_multiple_tickers(tickers)

# Save to files
scraper.save_to_csv(all_articles)
scraper.save_to_json(all_news)
    """)


EXAMPLE 1: Single Ticker (AAPL)
Fetching RSS feed for AAPL...
  ✓ Found 20 articles for AAPL

Displaying 3 of 20 articles:

1. Huge AI Deals Keep Markets at Record Highs
   Ticker: AAPL
   Published: 2025-09-22 22:18:00
   Summary: NVIDIA is investing $100 billion into ChatGPT-parent OpenAI's data centers.
   Link: https://finance.yahoo.com/news/huge-ai-deals-keep-markets-221800916.html?.tsrc=rss

2. Tigress Financial Partners Raises Its Price Target on Apple Inc. (AAPL) to $305
   Ticker: AAPL
   Published: 2025-09-22 22:06:04
   Summary: Apple Inc. (NASDAQ:AAPL) is one of the 13 Best Virtual Reality Stocks to Buy Right Now. On September 17, 2025, Tigress Financial Partners raised its price target on Apple Inc. (NASDAQ:AAPL) to $305, keeping a ‘Strong Buy’ rating. The bullish stance stems from the company’s accelerating services growth, aggressive AI innovation, and stronger U.S. supply […]
   Link: https://finance.yahoo.com/news/tigress-financial-partners-raises-price-220604882.html

In [ ]:
# Import the required libraries
import feedparser
import pandas as pd
from datetime import datetime
import json
import time

# Copy the YahooFinanceRSSScraper class from the file
# (or save the file as yahoo_rss_scraper.py and import it)

# Initialize the scraper
scraper = YahooFinanceRSSScraper()

# Get news for AAPL
apple_news = scraper.get_news_for_ticker('AAPL')

# Display the results (no truncation - full content)
scraper.display_articles(apple_news, max_display=10, truncate_desc=False)

# Or print each article manually for custom formatting
print(f"\nFound {len(apple_news)} articles for AAPL\n")
print("="*80)

for i, article in enumerate(apple_news, 1):
    print(f"\nArticle {i}:")
    print(f"Title: {article['title']}")
    print(f"Published: {article.get('published_formatted', article['published'])}")
    print(f"Description: {article['description']}")
    print(f"Link: {article['link']}")
    print("-"*80)

# Save to CSV for spreadsheet viewing
scraper.save_to_csv(apple_news, 'aapl_news.csv')

# Save to JSON for programmatic use
scraper.save_to_json(apple_news, 'aapl_news.json')

# Access the raw data as a list of dictionaries
for article in apple_news:
    # Each article is a dictionary with these keys:
    # 'ticker', 'title', 'link', 'description', 'guid',
    # 'published', 'published_datetime', 'published_formatted', 'scraped_at'
    print(article['title'])

Fetching RSS feed for AAPL...
  ✓ Found 20 articles for AAPL

Displaying 10 of 20 articles:

1. Huge AI Deals Keep Markets at Record Highs
   Ticker: AAPL
   Published: 2025-09-22 22:18:00
   Summary: NVIDIA is investing $100 billion into ChatGPT-parent OpenAI's data centers.
   Link: https://finance.yahoo.com/news/huge-ai-deals-keep-markets-221800916.html?.tsrc=rss

2. Tigress Financial Partners Raises Its Price Target on Apple Inc. (AAPL) to $305
   Ticker: AAPL
   Published: 2025-09-22 22:06:04
   Summary: Apple Inc. (NASDAQ:AAPL) is one of the 13 Best Virtual Reality Stocks to Buy Right Now. On September 17, 2025, Tigress Financial Partners raised its price target on Apple Inc. (NASDAQ:AAPL) to $305, keeping a ‘Strong Buy’ rating. The bullish stance stems from the company’s accelerating services growth, aggressive AI innovation, and stronger U.S. supply […]
   Link: https://finance.yahoo.com/news/tigress-financial-partners-raises-price-220604882.html?.tsrc=rss

3. UBS Reiterates Ne

In [2]:
!pip install feedparser

In [ ]:
# ============================================
# FULL ARTICLE CONTENT EXTRACTION
# Extends RSS scraper to fetch complete article text
# ============================================

import feedparser
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import re

class YahooFinanceFullArticleScraper:
    """
    Extracts full article content from Yahoo Finance RSS feeds
    Uses RSS for article discovery, then fetches full content from article URLs
    """

    def __init__(self):
        self.base_rss_url = "https://feeds.finance.yahoo.com/rss/2.0/headline?s={}&region=US&lang=en-US"
        self.headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}

    def _extract_article_text(self, url):
        """Extract full article text from a Yahoo Finance article URL"""
        try:
            response = requests.get(url, headers=self.headers, timeout=10)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            # Find article content - Yahoo Finance uses various selectors
            article_content = None

            # Try common article content selectors
            selectors = [
                'article',
                '[data-module="ArticleBody"]',
                '.caas-body',
                '.article-body',
                '[class*="article"]',
                '[class*="content"]'
            ]

            for selector in selectors:
                article_content = soup.select_one(selector)
                if article_content:
                    break

            if not article_content:
                # Fallback: find main content area
                article_content = soup.find('main') or soup.find('article')

            if article_content:
                # Remove script and style elements
                for script in article_content(["script", "style", "nav", "footer", "header"]):
                    script.decompose()

                # Extract text and clean it
                text = article_content.get_text(separator=' ', strip=True)
                # Clean up multiple whitespaces
                text = re.sub(r'\s+', ' ', text).strip()
                return text

            return None

        except Exception as e:
            return None

    def get_full_articles_for_ticker(self, ticker, max_articles=10, verbose=True):
        """
        Get full article content for a ticker symbol

        Args:
            ticker (str): Stock ticker symbol
            max_articles (int): Maximum number of articles to fetch
            verbose (bool): Print progress messages

        Returns:
            list: List of articles with full text content
        """
        articles = []

        if verbose:
            print(f"Fetching RSS feed for {ticker}...")

        try:
            # Get RSS feed
            feed_url = self.base_rss_url.format(ticker.upper())
            feed = feedparser.parse(feed_url)

            if not feed.entries:
                if verbose:
                    print(f"  ⚠ No articles found for {ticker}")
                return articles

            if verbose:
                print(f"Found {len(feed.entries)} articles in RSS feed")
                print(f"Fetching full content (filtering for articles >150 words)...\n")

            # Process articles until we have max_articles that meet the word count requirement
            articles_processed = 0
            articles_skipped = 0
            
            for entry in feed.entries:
                # Stop if we have enough articles
                if len(articles) >= max_articles:
                    break
                
                articles_processed += 1
                article_url = entry.get('link', '').strip()
                title = entry.get('title', '').strip()

                if not article_url:
                    continue

                if verbose:
                    title_short = title[:60] + "..." if len(title) > 60 else title
                    print(f"[{articles_processed}] Fetching: {title_short}...")

                # Extract full article text
                full_text = self._extract_article_text(article_url)

                # Parse publication date
                published = entry.get('published', '')
                published_datetime = None
                if hasattr(entry, 'published_parsed') and entry.published_parsed:
                    try:
                        published_datetime = datetime(*entry.published_parsed[:6])
                    except:
                        pass

                # Calculate word count
                word_count = len(full_text.split()) if full_text else 0

                # Filter: Only keep articles with more than 150 words
                if word_count <= 150:
                    articles_skipped += 1
                    if verbose:
                        print(f"    ⚠ Skipped: {word_count} words (minimum 150 required)")
                    continue

                article = {
                    'ticker': ticker.upper(),
                    'title': title,
                    'link': article_url,
                    'rss_description': entry.get('summary', '').strip(),
                    'published': published,
                    'published_datetime': published_datetime,
                    'guid': entry.get('guid', ''),
                    'full_text': full_text or '',
                    'word_count': word_count,
                    'has_full_text': full_text is not None and len(full_text) > 0
                }

                articles.append(article)

                if verbose and full_text:
                    print(f"    ✓ Retrieved {word_count} words")
                elif verbose:
                    print(f"    ⚠ Could not extract content")

            # Summary
            if verbose:
                print(f"\n📊 Summary:")
                print(f"   Articles processed: {articles_processed}")
                print(f"   Articles skipped (<150 words): {articles_skipped}")
                print(f"   Articles returned: {len(articles)}")

            return articles

        except Exception as e:
            if verbose:
                print(f"  ✗ Error: {e}")
            return articles

# ============================================
# USAGE EXAMPLE
# ============================================

# Initialize scraper
scraper = YahooFinanceFullArticleScraper()

# Get full articles for a ticker
articles = scraper.get_full_articles_for_ticker('AAPL', max_articles=3)

# Display results
print("\n" + "="*80)
print("FULL ARTICLE CONTENT")
print("="*80)

for i, article in enumerate(articles, 1):
    print(f"\nArticle {i}: {article['title']}")
    print(f"Published: {article['published']}")
    print(f"Word Count: {article['word_count']}")
    print(f"Has Full Text: {article['has_full_text']}")
    if article['full_text']:
        preview = article['full_text'][:200] + "..." if len(article['full_text']) > 200 else article['full_text']
        print(f"\nPreview:\n{preview}")
    print("-"*80)


Fetching RSS feed for AAPL...
Found 20 articles in RSS feed
Fetching full content for up to 3 articles...

[1/3] Fetching: Nvidia’s results ease concerns over AI boom...
    ✓ Retrieved 618 words
[2/3] Fetching: Apple Stock Has Made Investors Rich for 20 Years — What Happ......
    ✓ Retrieved 622 words
[3/3] Fetching: Nvidia's earnings attest to its leadership in the AI race. B......
    ✓ Retrieved 419 words

FULL ARTICLE CONTENT

Article 1: Nvidia’s results ease concerns over AI boom
Published: Wed, 19 Nov 2025 22:23:41 +0000
Word Count: 618
Has Full Text: True

Preview:
Nvidia’s results ease concerns over AI boom Michael Liedtke, Associated Pres Wed 19 November 2025 at 5:23 pm GMT-5 3 min read NVDA AAPL Nvidia’s sales of the computing chips powering artificial intell...
--------------------------------------------------------------------------------

Article 2: Apple Stock Has Made Investors Rich for 20 Years — What Happens Next?
Published: Wed, 19 Nov 2025 21:55:08 +0000
Word Coun

In [13]:
# ============================================
# QUICK USAGE: Extract Full Article Content
# ============================================

# Use the scraper from Cell 4
scraper = YahooFinanceFullArticleScraper()

# Get full articles for a ticker
articles = scraper.get_full_articles_for_ticker('AAPL', max_articles=5)

# Access full article content
for article in articles:
    print(f"\n{'='*80}")
    print(f"Title: {article['title']}")
    print(f"Word Count: {article['word_count']}")
    print(f"Link: {article['link']}")
    print(f"\nFull Text:\n{article['full_text'][:500]}..." if len(article['full_text']) > 500 else f"\nFull Text:\n{article['full_text']}")
    print(f"{'='*80}")

Fetching RSS feed for AAPL...
Found 20 articles in RSS feed
Fetching full content for up to 5 articles...

[1/5] Fetching: Nvidia’s results ease concerns over AI boom...
    ✓ Retrieved 620 words
[2/5] Fetching: Apple Stock Has Made Investors Rich for 20 Years — What Happ......
    ✓ Retrieved 624 words
[3/5] Fetching: Nvidia's earnings attest to its leadership in the AI race. B......
    ✓ Retrieved 419 words
[4/5] Fetching: Stocks steadier before key Nvidia results as oil slides...
    ✓ Retrieved 655 words
[5/5] Fetching: Alphabet Stock Rises, but Falls Short of Passing Microsoft i......
    ✓ Retrieved 148 words

Title: Nvidia’s results ease concerns over AI boom
Word Count: 620
Link: https://uk.finance.yahoo.com/news/nvidia-results-ease-concerns-over-215114015.html?.tsrc=rss

Full Text:
Nvidia’s results ease concerns over AI boom Michael Liedtke, Associated Pres Wed 19 November 2025 at 5:23 pm GMT-5 3 min read NVDA +2.85% AAPL +0.42% Nvidia’s sales of the computing chips powering 

In [22]:
articles[0]['full_text']

'Nvidia’s results ease concerns over AI boom Michael Liedtke, Associated Pres Wed 19 November 2025 at 5:23 pm GMT-5 3 min read NVDA +2.85% AAPL +0.42% Nvidia’s sales of the computing chips powering artificial intelligence have surged beyond the lofty bar set by stock market analysts. The performance may ease recent jitters about a Big Tech boom turning into a bust that topples the world’s most valuable company. The results announced late on Wednesday provided an update on the frenzied spending on AI technology that has been fuelling both the stock market and much of the overall US economy since OpenAI released ChatGPT three years ago. Nvidia has been by far the biggest beneficiary of the run-up because its processors have become indispensable for building the AI factories that are needed to enable what is supposed to be the most dramatic shift in technology since Apple released the iPhone in 2007. But in the past few weeks there has been a rising tide of sentiment that the high expecta